In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [2]:
train_matches = pd.read_csv('train.csv')
test_matches = pd.read_csv('test.csv')
gold = pd.read_csv('gold.csv')

Как признаки будем использовать количество денег, которое игроки успели получить к 10-ой минуте.

In [3]:
gold = gold[gold.times == 600]

Просуммируем деньги для каждой команды.

In [4]:
radiant_gold = gold[['player_0', 'player_1', 'player_2', 'player_3', 'player_4']].sum(axis=1)
dire_gold = gold[['player_5', 'player_6', 'player_7', 'player_8', 'player_9']].sum(axis=1)

gold['radiant_gold'] = radiant_gold
gold['dire_gold'] = dire_gold

Сделаем несколько простых признаков: разницу суммы денег и отношение.

In [5]:
gold['diff_gold'] = gold['radiant_gold'] - gold['dire_gold']
gold['ratio_gold'] = gold['radiant_gold'] / gold['dire_gold']

Склеим всё с нужными играми из train и test.

In [6]:
train = pd.merge(train_matches[['mid']], gold, on='mid', how='left').drop(['mid', 'times'], 1)
test = pd.merge(test_matches[['mid']], gold, on='mid', how='left').drop(['mid', 'times'], 1)

Зафиксируем что является признаками и таргетом.

In [7]:
x_train = train.values
x_test = test.values
y_train = train_matches.radiant_won.values

В качестве модели будем использовать RandomForestClassifier со 100 деревьями.

In [8]:
clf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=1234)
np.mean(cross_val_score(clf, x_train, y_train, cv=5, scoring='roc_auc'))

0.6859411256907274

Обучимся на полных данных.

In [9]:
clf.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=1234, verbose=0,
            warm_start=False)

Добавим новую колонку с предсказаниями в файл с тестовыми идентификаторами.

In [10]:
test_matches['radiant_won'] = clf.predict_proba(x_test)[:, 1]

И запишем в файл для отправки в Kaggle

In [11]:
test_matches.to_csv('baseline.csv', index=None)